In [36]:
import threading
from pathlib import Path

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [37]:
from pyspark.streaming import StreamingContext

In [38]:
sc  # at this point Spark is such a blackbox to me. I'll just leave this here

<SparkContext master=local[*] appName=PySparkShell>

In [39]:
ssc = StreamingContext(sc, 10)
lines = ssc.socketTextStream("seppe.net", 7778)

In [40]:
# Change here if you would like to generate more data
path = Path()
path = path.joinpath("data_workaround", "data")
lines.repartition(1).saveAsTextFiles(path.absolute().as_uri())  # absolute path, as_uri is needed

In [41]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

-------------------------------------------
Time: 2023-05-06 16:51:40
-------------------------------------------
{"review_id": "137880693", "app_id": "2086140", "review_text": "I have already seen enough!\nThis game is beyond bad in multiple aspects.\n- The performance is so terrible when you try turning bloom/shader effects on or even at higher settings  - it will melt your GPU while still looking very simple XDDDDD. \n- Upgrades are unrecognizeable and some do the same and just have a different name (means you have to read through every upgrade on every level up and do that probably for the whole run and beyond).\n- Hit feedback when receiving damage is basicaly non-exsistent. Occacionaly you look over to your healthbar to the right border of your screen and be like - oh when did my health get soo low and how??!!.\n\n2 Bucks is not much money asked and there is content but its NOT worth your time at version 20 right now!\nIt has potential but also a loong way ahead of itself and i d

In [42]:
# stop thread
ssc_t.stop()

----- Stopping... this may take a few seconds -----
-------------------------------------------
Time: 2023-05-06 16:52:30
-------------------------------------------

-------------------------------------------
Time: 2023-05-06 16:52:40
-------------------------------------------



In [217]:
def get_data_paths():
    path = Path()

    saved_file_generator = path.joinpath("data_workaround").rglob("**/part-*")
    paths = [str(path.absolute()) for path in saved_file_generator]
    
    return paths
get_data_paths()  # nothing is hard coded, so don't worry about C:\\Lunky...

['C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384700000\\part-00000',
 'C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384710000\\part-00000',
 'C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384720000\\part-00000',
 'C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384730000\\part-00000',
 'C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384740000\\part-00000',
 'C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384750000\\part-00000',
 'C:\\Users\\Lunky\\Desktop\\spark\\data_workaround\\data-1683384760000\\part-00000']

In [213]:
def turn_data_into_df():
    """
    When you run letsgo-win.bat, Seppe have setup variables "spark" and "sc".
    Variable "spark" is of the type spark: pyspark.sql.session.SparkSession
    
    Returns an object of the type spark DataFrame
    """
    path = Path()

    saved_file_generator = path.joinpath("data_workaround").rglob("**/part-*")
    paths = [str(path.absolute()) for path in saved_file_generator]
    
    return spark.read.json(paths)

In [216]:
df = turn_data_into_df()
df.show()

+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1155330|    1|137878506|I am a huge fan o...|
|1669000|    1|137881705|The game is fun. ...|
|1669000|    1|137881680|          great game|
|1669000|    1|137881642|I finally decided...|
|2086140|    0|137880693|I have already se...|
|2246110|    1|137881266|At several moment...|
|1277920|    0|137880853|Classes are unbal...|
|1294810|    0|137881041|     Jesus Christ...|
|1294810|    1|137878553|      Hear me out...|
|1294810|    0|137878208|The game is just ...|
+-------+-----+---------+--------------------+

